In [ ]:
#Se hará un modelo capaz de reconocer conductas inapropiadas 
"""_summary_
        Implemtaciones posibles:
            Tooling: 
                Chatbot? Míneria de textos, de alguna red social, gmail, sistema de control?
            Institucional: 
                Sería útil para las empresas en detectar conductas inapropiadas tales como racismo,
                machismo, acoso sexual, por medios oficiales de comunicación.
                    datasets = machismo, acoso sexual, racismo, xenofobia
                Implementado como un sistema de control parental, para controlar textos peligrosos
                    datasets? 
    """

'_summary_\n        Implemtaciones posibles:\n            Tooling: \n                Chatbot? Míneria de textos, de alguna red social, gmail, sistema de control?\n            Institucional: \n                Sería útil para las empresas en detectar conductas inapropiadas tales como racismo,\n                machismo, acoso sexual, por medios oficiales de comunicación.\n                    datasets = machismo, acoso sexual, racismo\n                Implementado como un sistema de control parental, para controlar textos peligrosos\n                    datasets? \n    '

In [2]:
# Importes
import kagglehub
import os
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from mlxtend.plotting import plot_confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt

/home/jordi/entornos/entorno_sic/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#Acá irán los datasets, ahorita en concepto de uso


"""
    Es un dataset que nos permite detectar textos racistas, en base al target:
    target inicial {1:is racist, 0:non-racist}
"""
path = kagglehub.dataset_download("rishabhjohri/racismdetectiondataset")
print("Path to dataset files:", path)

Path to dataset files: /home/jordi/.cache/kagglehub/datasets/rishabhjohri/racismdetectiondataset/versions/1


In [3]:
df_racism = pd.read_csv('../data_sets/behaviors_data/RacismDetectionDataSet.csv')
rows , columns = df_racism.shape
print(f'Filas: {rows}\nColumnas: {columns}')
df_racism.head()

Filas: 1999
Columnas: 2


,Comment,Label
0,i was born a racist and I will die a racist I ...,1
1,bitch nigga miss me with that,1
2,if you aint bout that murder game pussy nigga ...,1
3,gay niggas couldnt wait to act like bitches to...,1
4,why deos a gorilla always have a frown because...,1


In [ ]:
# Modificamos las etiquetas, y las columnas
# text -- emotion

df_racism = df_racism.rename(columns={'Comment':'text', 'Label':'emotion'})
df_racism['emotion'] = df_racism['emotion'].replace({1:'racist',2:'non-racist'})
 #Quitaremos los 0, para el modelo ya entrenado en neutral?
 

,text,emotion
0,i was born a racist and I will die a racist I ...,racist
1,bitch nigga miss me with that,racist
2,if you aint bout that murder game pussy nigga ...,racist
3,gay niggas couldnt wait to act like bitches to...,racist
4,why deos a gorilla always have a frown because...,racist
...,...,...
1994,The federal government deliberately targeted B...,0
1995,A reminder we never got a foreign policy debat...,0
1996,floyd maywhether and jake paul single handedly...,0
1997,We know the name of the cop who killed George ...,0


In [13]:
"""_summary_
    Este dataset nos permite identificar bullyng en comentarios
    """

path = kagglehub.dataset_download("saifulislam7/cyberbullying-and-harassment-detection-using-ml")

print("Path to dataset files:", path)
df_discg = pd.read_csv('../data_sets/behaviors_data/discrimination_variety.csv')
df_discg

Path to dataset files: /home/jordi/.cache/kagglehub/datasets/saifulislam7/cyberbullying-and-harassment-detection-using-ml/versions/1


,Text,Label,Types
0,Ten outside soon doctor shake everyone treatme...,Not-Bullying,NaN
1,my life has come to a standstill and at this p...,Not-Bullying,NaN
2,girl this nigga make me sick to my stomach,Bullying,Ethnicity
3,I wanna fuck you,Bullying,Sexual
4,"Oh hey, you should be ashamed of your disgusti...",Not - Bullying,NaN
...,...,...,...
8447,Please help this Christian nationalist battle ...,Bullying,Vocational
8448,Has the Holy Quran ever been read by the membe...,Bullying,Sexual
8449,And that is the difficulty - you need basic gu...,Bullying,Religion
8450,"As they will only encounter Muslim women, help...",Bullying,Religion


In [ ]:
rows, colums = df_discg.shape
print(f'Filas: {rows}\nColumnas: {columns}')
print(df_discg['Types'].unique())

Filas: 8452
Columnas: 2
[nan 'Ethnicity' 'Sexual' 'Religion ' 'Religion' 'Threats ' 'Troll'
 'Troll ' 'sexual' 'Vocational' 'Threats' 'Vocational ' 'Political'
 'Ethnicity ' 'Religious ' 'Sexual ' 'Political ' 'Threat' 'Racism'
 'Vocation' 'Saxual' 'political ' 'Religon' 'Ethnically ' 'Threat ']


In [41]:
#Limpieza -> uniformidad de datos
df_discg = df_discg.rename(columns={'Text':'text','Types':'emotion'})
df_discg = df_discg.drop('Label', axis=1)
df_discg


,text,emotion
0,Ten outside soon doctor shake everyone treatme...,NaN
1,my life has come to a standstill and at this p...,NaN
2,girl this nigga make me sick to my stomach,Ethnicity
3,I wanna fuck you,Sexual
4,"Oh hey, you should be ashamed of your disgusti...",NaN
...,...,...
8447,Please help this Christian nationalist battle ...,Vocational
8448,Has the Holy Quran ever been read by the membe...,Sexual
8449,And that is the difficulty - you need basic gu...,Religion
8450,"As they will only encounter Muslim women, help...",Religion


In [5]:
"""_summary_
    Acoso sexual, etiquetas explícitas
    """

path = kagglehub.dataset_download("aadyasingh55/sexism-detection-in-english-texts")
print("Path to dataset files:", path)

100%|██████████| 1.26M/1.26M [00:00<00:00, 4.00MB/s]

Extracting files...
Path to dataset files: /home/jordi/.cache/kagglehub/datasets/aadyasingh55/sexism-detection-in-english-texts/versions/1


In [36]:
df_sexh1 = pd.read_csv('../data_sets/behaviors_data/sexual_harassment1.csv')
df_sexh2 = pd.read_csv('../data_sets/behaviors_data/sexual_harassment2.csv')
df_sexh3 = pd.read_csv('../data_sets/behaviors_data/sexual_harassment3.csv')
df_sexh1

df_sexh = pd.concat([df_sexh1,df_sexh2,df_sexh3], ignore_index=True)
df_sexh

,rewire_id,text,label_sexist,label_category,label_vector,split
0,sexism2022_english-9609,"In Nigeria, if you rape a woman, the men rape ...",not sexist,none,none,dev
1,sexism2022_english-966,I bet she wished she had a gun,not sexist,none,none,dev
2,sexism2022_english-3553,I agree with that but at the same time I know ...,sexist,2. derogation,2.1 descriptive attacks,dev
3,sexism2022_english-13328,democrats from Minnesota (leftist muzzie shith...,sexist,2. derogation,2.2 aggressive and emotive attacks,dev
4,sexism2022_english-81,Texas jury sentences Jordanian immigrant to de...,not sexist,none,none,dev
...,...,...,...,...,...,...
19995,sexism2022_english-12034,I blew a shot at a record like that. Girl on t...,not sexist,none,none,train
19996,sexism2022_english-5228,girls really get fucked almost every weekend ?,not sexist,none,none,train
19997,sexism2022_english-10140,The hatred for moslems are Reasonable and Just...,not sexist,none,none,train
19998,sexism2022_english-9726,Now this is a woman who gets it. 👆,not sexist,none,none,train


In [37]:
df_sexh = df_sexh.drop('rewire_id',axis=1)
df_sexh = df_sexh.drop('label_category',axis=1)
df_sexh = df_sexh.drop('label_vector',axis=1)
df_sexh = df_sexh.drop('split',axis=1)
df_sexh = df_sexh[df_sexh['label_sexist'] != 'not sexist']

df_sexh

,text,label_sexist
2,I agree with that but at the same time I know ...,sexist
3,democrats from Minnesota (leftist muzzie shith...,sexist
7,aim for the head so she dosn't come back,sexist
9,This guys gonna have a great time in the barra...,sexist
13,Or women can stop being lying whores.,sexist
...,...,...
19968,The drama queen is just another open borders c...,sexist
19985,This is easily the dumbest thing ever written....,sexist
19986,Recently some thots started advertizing their ...,sexist
19988,I wish yall understand what a burden it can be...,sexist


In [38]:
#Rename
df_sexh = df_sexh.rename(columns={'label_sexist':'emotion'})
rows, columns = df_sexh.shape
print(f'Filas: {rows}\nColumnas: {columns}')
print(df_sexh['emotion'].unique())

Filas: 4854
Columnas: 2
['sexist']


In [42]:
# Concatenar todos los dataSets en uno solo
df = pd.concat([ df_discg, df_racism,df_sexh], ignore_index=True)
rows, columns = df.shape
df

,text,emotion
0,Ten outside soon doctor shake everyone treatme...,NaN
1,my life has come to a standstill and at this p...,NaN
2,girl this nigga make me sick to my stomach,Ethnicity
3,I wanna fuck you,Sexual
4,"Oh hey, you should be ashamed of your disgusti...",NaN
...,...,...
15300,The drama queen is just another open borders c...,sexist
15301,This is easily the dumbest thing ever written....,sexist
15302,Recently some thots started advertizing their ...,sexist
15303,I wish yall understand what a burden it can be...,sexist


In [ ]:
#Diccionarios de reglas?
#Arquitectura, SVM, RedNeuronal?    
